# Introduction

[Start here](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts)

# Key Concepts and Terminology

MCP is often described as the "USC=C for AI applications." Just as USB-C provides a standardized physical and logical interface for connecting various peripherals to computing devices, MCP offers a consistent protocol for linking AI models to external capabilities. This standardization benefits the entire ecosystem:
* users enjoy simpler and more consistent experiences across AI applications
* AI application developers gain easy integration with a growing ecosystem of tools and data sources
* tool and data providers need only create a single implementation that works with multiple AI applications
* the broader ecosystem benefits from increased interoperability, innovation, and reduced fragmentation

## The Integration Problem

The **MxN Integration Problem** refers to the challenge of connecting M different AI applications to N different external tools or data sources without a standardized approach.

## Without MCP (M×N Problem)

Without a protocol like MCP, developers would need to create M×N custom integrations—one for each possible pairing of an AI application with an external capability.

Each AI application would need to integrate with each tool/data source individually. This is a very complex and expensive process which introduces a lot of friction for developers, and high maintenance costs.

Once we have multiple models and multiple tools, the number of integrations becomes too large to manage, each with its own unique interface.

## With MCP (M+N Solution)

MCP transforms this into an M+N problem by providing a standard interface: each AI application implements the client side of MCP once, and each tool/data source implements the server side once. This dramatically reduces integration complexity and maintenance burden.

## Core MCP Terminology

MCP is a standard like HTTP or USB-C, and is a protocol for connecting AI applications to external tools and data sources. Therefore, using standard terminology is crucial to making the MCP work effectively.

When documenting our applications and communicating with the community, we should use the following terminology.

Just like client server relationships in HTTP, MCP has a client and a server.

* **Host:** The user-facing AI application that end-users interact with directly. Examples include Anthropic’s Claude Desktop, AI-enhanced IDEs like Cursor, inference libraries like Hugging Face Python SDK, or custom applications built in libraries like LangChain or smolagents. Hosts initiate connections to MCP Servers and orchestrate the overall flow between user requests, LLM processing, and external tools.

* **Client:** A component within the host application that manages communication with a specific MCP Server. Each Client maintains a 1:1 connection with a single Server, handling the protocol-level details of MCP communication and acting as an intermediary between the Host’s logic and the external Server.

* **Server:** An external program or service that exposes capabilities (Tools, Resources, Prompts) via the MCP protocol.

[Terminology](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts#capabilities)

**Then there's a bunch of other stuff you're better off just reading instead of typing along to.**

# The Communication Protocol

[Resource](https://huggingface.co/learn/mcp-course/en/unit1/communication-protocol)

MCP defines a standardized communication protocol that enables Clients and Servers to exchange messages in a consistent, predictable way. This standardization is critical for interoperability across the community.

## JSON-RPC: The Foundation

At its core, MCP uses **JSON-RPC 2.0** as the message format for all communication between clients and servers. JSON-RPC is a lightweight remote procedure call protocol encoded in JSON, which makes it:
* Human-readable and easy to debug
* Language-agnostic, supporting implementation in any programming environment
* Well-established, with clear specifications and widespread adoption

The protocol defines three types of messages:

### Requests

Sent from Client to Server to initiate an operation. A request message includes:
* A unique ID (`id`)
* The method name to invoke (e.g. `tools/call`)
* Parameters for the method (if any)

Example request:

In [2]:
import json
import requests

request = {
    'jsonrpc': '2.0',
    'id': 1,
    'params' : {
        'name': 'weather',
        'arguments': {
            'location': 'San Francisco'
        }
    }
}

json_string = json.dumps(request)
json_string_pretty = json.dumps(request, indent=2)
print(json_string_pretty)

{
  "jsonrpc": "2.0",
  "id": 1,
  "params": {
    "name": "weather",
    "arguments": {
      "location": "San Francisco"
    }
  }
}


### Responses

Sent from Server to Client in reply ot a Request. A Response message includes:
* The same `id` as the corresponding Request
* Either a `result` (for success) or and `error` (for failure)

Example Success Response:

In [3]:
response = {
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "temperature": 62,
    "conditions": "Partly cloudy"
  }
}

json_string = json.dumps(response)
json_string_pretty = json.dumps(response, indent=2)
print(json_string_pretty)

{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "temperature": 62,
    "conditions": "Partly cloudy"
  }
}


Example Error Response:

In [4]:
{
  "jsonrpc": "2.0",
  "id": 1,
  "error": {
    "code": -32602,
    "message": "Invalid location parameter"
  }
}

{'jsonrpc': '2.0',
 'id': 1,
 'error': {'code': -32602, 'message': 'Invalid location parameter'}}

### Notifications
One-way messages that don’t require a response. Typically sent from Server to Client to provide updates or notifications about events.

Example Notification:

In [5]:
{
  "jsonrpc": "2.0",
  "method": "progress",
  "params": {
    "message": "Processing data...",
    "percent": 50
  }
}

{'jsonrpc': '2.0',
 'method': 'progress',
 'params': {'message': 'Processing data...', 'percent': 50}}

## Transport Mechanisms

JSON-RPC defines the message format, but MCP also specifies how these messages are transported between Clients and Servers. Two primary transport mechanisms are supported.

### stdio (Standard Input/Output)

The stdio transport is used for local communication, where the Client and Server run on the same machine:

The Host application launches the Server as a subprocess and communicates with it by writing to its standard input (stdin) and reading from its standard output (stdout).

Use cases for this transport are local tools like file system access or running local scripts.

The main Advantages of this transport are that it’s simple, no network configuration required, and securely sandboxed by the operating system.

### HTTP + SSE (Server-Sent Events) / Streamable HTTP

The HTTP+SSE transport is used for remote communication, where the Client and Server might be on different machines:

Communication happens over HTTP, with the Server using Server-Sent Events (SSE) to push updates to the Client over a persistent connection.

Use cases for this transport are connecting to remote APIs, cloud services, or shared resources.

The main Advantages of this transport are that it works across networks, enables integration with web services, and is compatible with serverless environments.

Recent updates to the MCP standard have introduced or refined “Streamable HTTP,” which offers more flexibility by allowing servers to dynamically upgrade to SSE for streaming when needed, while maintaining compatibility with serverless environments.

Find notes on the **Interaction Lifecycle** [here](https://huggingface.co/learn/mcp-course/en/unit1/communication-protocol#the-interaction-lifecycle).

# Understanding MCP Capabilities

MCP Servers expose a variety of capabilities to Clients through the communication protocol. These capabilities fall into four main categories.

## Tools

Tools are executable functions or actions that the AI model can invoke through the MCP protocol
* **Control:** Tools are typically **model-controlled**, meaning that the LLM decides when to call them based on the user's request and context.
* **Safety:** Due to their ability to perform actions with side effects, tool execution can be dangerous. Therefore, they typically require explicit user approval.
* **Use Cases:** Sending messages, creating tickets, querying APIs, performing calculations.

**Example:** A weather tool that fetches current weather data for a given location:

In [6]:
def get_weather(location: str) -> dict:
    """Get the current weather for a specified location."""
    return {
        "temperature": 72,
        "conditions": "Sunny",
        "humidity": 45
    }

## Resources

Resources provide read-only access to data sources, allowing the AI model to retrieve context without executing complex logic.
* **Control:** Resources are **application-controlled**, meaning the Host application typically decides when to access them.
* **Nature:** They are designed for data retrieval with minimal computation, similar to GET endpoints in REST APIs.
* **Safety:** Since they are read-only, they typically present lower security risks than Tools.
* **Use Cases:** Accessing file contents, retrieving database records, reading configuration information.

**Example:** A resource that provides access to file contents:

In [7]:
def read_file(file_path: str) -> str:
    """Read the contents of a file at the specified path."""
    with open(file_path, 'r') as f:
        return f.read()

## Prompts

Prompts are predefined templates or workflows that guide the interaction between the user, the AI model, and the Server's capabilities.
* **Control:** Prompts are user-controlled, often presented as options in the Host application's UI.
* **Purpose:** They structure interactions for optimal use of available Tools and Resources.
* **Selection:** Users typically select a prompt before the AI model begins processing, setting context for the interaction.
* **Use Cases:** Common workflows, specialized task templates, guided interactions.

**Example:** A prompt template for generating a code review:

In [8]:
def code_review(code: str, language: str) -> list:
    """Generate a code review for the provided code snippet."""
    return [
        {
            "role": "system",
            "content": f"You are a code reviewer examining {language} code. Provide a detailed review highlighting best practices, potential issues, and suggestions for improvement."
        },
        {
            "role": "user",
            "content": f"Please review this {language} code:\n\n```{language}\n{code}\n```"
        }
    ]

## Sampling

Sampling allows Servers to request the Client (specifically, the Host application) to perform LLM interactions.
* **Control:** Sampling is server-initiated, but requires Client/Host facilitation.
* **Purpose:** It enables server-driven agentic behaviors and potentially recursive or multi-step interactions.
* **Safety:** Like Tools, sampling operations typically require user approval.
* **Use Cases:**. Complex multi-step tasks, autonomous agent workflows, interactive processes.

**Example:** A Server might request the Client to analyze data it has processed:

In [9]:
def request_sampling(messages, system_prompt=None, include_context="none"):
    """Request LLM sampling from the client."""
    # In a real implementation, this would send a request to the client
    return {
        "role": "assistant",
        "content": "Analysis of the provided data..."
    }

The sampling flow follows these steps:
1. Server sends a sampling/createMessage request to the client
1. Client reviews the request and can modify it
1. Client samples from an LLM
1. Client reviews the completion
1. Client returns the result to the server

Refer to [this page](https://huggingface.co/learn/mcp-course/en/unit1/capabilities?resource-example=python&prompt-example=python#how-capabilities-work-together) for information on how these capabilities work together.

# Discovery Process

One of MCP's key features is dynamic capability discovery. hen a Client connects to a server, it can query the available Tools, Resources, and Prompts through specific list methods:
* `tools/list`: Discover available Tools
* `resources/list`: Discover available Resources
* `prompts/list`: Discover available Prompts

# MCP SDK

A Software Development Kit (SDK) helps software developers create applications for a specific platform, system, or programming language. Think of it kind of like a toolkit for app development. Typically, a basic SDK will include a compiler, debugger, and APIs.

The MCP provides official SDKs for both JavaScript, Python and other languages. This makes it easy to implement MCP clients and servers in applications. These SDKs handle the low-level protocol details.

## SDK Overview

Both SDKs provide similar core functionality, following the MCP protocol specification we discussed earlier. They handle:
* Protocol-level communication
* Capability registration and discovery
* Message serialization/deserialization
* Connection management
* Error handling

## Core Primitives Implementation

Now we're gonna explore how to implement each of the core primitives (Tools, Resources, and Prompts) using both SDKs.

(Also, I'm going back to using double quotes. I like double quotes.)

In [10]:
from mcp.server import FastMCP

mcp = FastMCP('Weather Service')

# Tool implementation
@mcp.tool()
def get_weather(location: str) -> str:
    """Get the current weather for a specified location."""
    return f"Weather in {location}: Sunny, 72°F"

# Resource implementation
@mcp.resource("weather://{location}")
def weather_resource(location: str) -> str:
    """Provide weather data as a resource."""
    return f"Weather data for {location}: Sunny, 72°F"

# Prompt implementation
@mcp.prompt()
def weather_report(location: str) -> str:
    """Create a weather report prompt"""
    return f"""You are a weather reporter. Weather report for {location}?"""

# Run the server
if __name__ == "__main__":
    mcp.run()

RuntimeError: Already running asyncio in this thread

Once you have your server implemented, you can start it running the server script.

```mcp dev server.py```

This will initialize a development server running the file `sever.py`. And log the following output:

```Starting MCP inspector...
⚙️ Proxy server listening on port 6277
Spawned stdio transport
Connected MCP client to backing server transport
Created web app transport
Set up MCP proxy
🔍 MCP Inspector is up and running at http://127.0.0.1:6274 🚀```

You can then open the MCP Inspector at http://127.0.0.1:6274 to see the server's capabilities and interact with them. This will bring you to a beautiful UI.

# MCP Clients

MCP CLients are crucial components that act as the bridge between AI applications (Hosts) and external capabilities provided by MCP Servers. Think of the Host as your main application (like an AI assistant or IDE) and the Client as a specialized module within that Host responsible for handling MCP communications.

# User Interface Client

## Chat Interface Clients
* Claude Desktop (Anthropic)

## Interactive Development Clients
* VS Code extensions with MCP
* Cursor IDE
* Zed editor

These clients support connecting to multiple MCP servers and real-time tool invocation.

## Configuring MCP Clients

Effective deployment of MCP servers and clients requires proper configuration. The MCP specification is still evolving, so the configuration methods are subject to evolution. We'll focus on the current best practices for configuration.

**MCP Configuration Files**

MCP hosts use configuration files to manage server connections. These files define which servers are available and how to connect to them.

Fortunately, the configuration files are very simple, easy to understand, and consistent across major MCP hosts.

**mcp.json Structure**

The standard configuration file for MCP is named `mcp.json`. Here's the basic structure:

In [ ]:
{
  "servers": [
    {
      "name": "Server Name",
      "transport": {
        "type": "stdio|sse",
        # Transport-specific configuration
      }
    }
  ]
}

In this example, we have a single server with a name and a transport type. The transport type is either `studio` or `sse`.

**Configuration for stdio Transport**

For local servers using stdio transport, the configuration includes the command and arguments to launch the server process:

In [ ]:
{
  "servers": [
    {
      "name": "File Explorer",
      "transport": {
        "type": "stdio",
        "command": "python",
        "args": ["/path/to/file_explorer_server.py"] // This is an example, we'll use a real server in the next unit
      }
    }
  ]
}

Here, we have a server called "File Explorer" that is a local script.

**Configuration for HTTP+SSE Transport**

For remote servers using HTTP+SSE transport, the configuration includes the server URL:

In [ ]:
{
  "servers": [
    {
      "name": "Remote API Server",
      "transport": {
        "type": "sse",
        "url": "https://example.com/mcp-server"
      }
    }
  ]
}

**Environment Variables Configuration**

In Python, we use the `os` module to access environment variables:

In [ ]:
import os

# Access environment variables
github_token = os.environ.get("GITHUB_TOKEN")
if not github_token:
    raise ValueError("GITHUB_TOKEN environment variable is required")

# Use the token in your server code
def make_github_request():
    headers = {"Authorization": f"Bearer {github_token}"}
    # ... rest of your code

The corresponding configuration in `mcp.json` would look like this:

```{
  "servers": [
    {
      "name": "GitHub API",
      "transport": {
        "type": "stdio",
        "command": "python",
        "args": ["/path/to/github_server.py"], // This is an example, we'll use a real server in the next unit
        "env": {
          "GITHUB_TOKEN": "your_github_token"
        }
      }
    }
  ]
}```

Go to [this link](https://huggingface.co/learn/mcp-course/en/unit1/mcp-clients#configuration-examples) for configuration examples.

# Tiny Agents Clients

Let's explore how to use MCP Clients within code.

You can also use tiny agents as MCP Clients to connect directly to MCP servers from your code. Tiny agents provide a simple way to create AI agents that can use tools from MCP servers.

Tiny Agent can run MCP servers with a command line environment.

## Setup

First, we will need to install `npx` with the following command:

```npm install -g npx```

Then, we will need to install the huggingface_hub package with the MCP support. This will allow us to run MCP servers and clients.

```pip install "huggingface_hub[mcp]>=0.32.0"```

Then, we will need to log in to the Hugging Face Hub to access the MCP servers. You can do this with the `huggingface-cli` command line tool. You will need a [login token](https://huggingface.co/docs/huggingface_hub/v0.32.3/en/quick-start#authentication) to do this.

```huggingface-cli login```

**Configure Access Token Permissions**

After creating your Hugging Face access token and logging in, you need to ensure your token has the proper permissions to work with inference providers.

1. Go to your [Hugging Face Access Tokens page](https://huggingface.co/settings/tokens)
1. Find your MCP token and click the three dots (⋮) next to it
1. Select “Edit permissions”
1. Under the Inference section, check the box for:
* “Make calls to Inference Providers”
1. Save your changes

This permission is required because tiny agents need to make API calls to hosted models like `Qwen/Qwen2.5-72B-Instruct` through providers like Nebius.

## Connecting to MCP Servers

Now, let's create an agent congfiguration file `agent.json`. You can find this file somewhere else (figure it out you fool).

Now you can run the agent:

```tiny-agents run agent.json```

In the [this video](), we run the agent and ask it to open a new tab in the browser.

(Seems they messed up with the documentation. I can't find the video anywhere.)

# Hugging Face MCP Server

The Hugging Face MCP Server connects your MCP-compatible AI assistant (for example VS Code, Cursor, Zed, or Claude Desktop) directly to the Hugging Face Hub. Once connected, your assistant can search and explore Hub resources and use community tools, all from within your editor, chat, or CLI.

**What you can do**

* Search and explore Hub resources: models, datasets, Spaces, and papers.
* Run community tools via MCP‑compatible Gradio apps hosted on Spaces.
* Bring results back into your assistant with metadata, links, and context.

**Built-in tools**

The server provides curated tools that work across supported clients:

* Models search and exploration (filter by task, library, downloads, likes)
* Datasets search and exploration (filter by tags, size, modality)
* Spaces semantic search (find apps by capability, e.g., TTS, ASR, OCR)
* Papers semantic search (discover relevant research on the Hub)

**Get started**

1. Open your MCP settings: visit https://huggingface.co/settings/mcp while logged in.
1. Pick your client: select your MCP‑compatible client (for example VS Code, Cursor, Zed, Claude Desktop). The page shows client‑specific instructions and a ready‑to‑copy configuration snippet.
1. Paste and restart: copy the snippet into your client’s MCP configuration, save, and restart/reload the client. You should see “Hugging Face” (or similar) listed as a connected MCP server in your client.

**Using the server**

After connecting, ask your assistance to use the Hugging Face tools.

Your assistant will call MCP tools exposed by the Hugging Face MCP Server. You can then open the resource on the Hub or continue iterating in the same chat.

# Gradio MCP Integration

Gradio is a popular Python library for quickly creating customizable web interfaces for machine learning models.

## Introduction to Gradio

Gradio allows developers to create UIs for their models with just a few lines of Python code. It’s particularly useful for:
* Creating demos and prototypes
* Sharing models with non-technical users
* Testing and debugging model behavior

With the addition of MCP support, Gradio now offers a straightforward way to expose AI model capabilities through the standardized MCP protocol.

Combining Gradio with MCP allows you to create both human-friendly interfaces and AI-accessible tools with minimal code. But best of all, Gradio is already well-used by the AI community, so you can use it to share your MCP Servers with others.

You’ll also need an LLM application that supports tool calling using the MCP protocol, such as Cursor ( known as “MCP Hosts”).

# Creating an MCP Server with Gradio

Let’s walk through a basic example of creating an MCP Server using Gradio:

In [11]:
import gradio as gr

def letter_counter(word: str, letter: str) -> int:
    """
    Count the number of occurrences of a letter in a word or text.

    Args:
        word (str): The input text to search through
        letter (str): The letter to search for

    Returns:
        int: The number of times the letter appears in the text
    """
    word = word.lower()
    letter = letter.lower()
    count = word.count(letter)
    return count

# Create a standard Gradio interface
demo = gr.Interface(
    fn=letter_counter,
    inputs=["textbox", "textbox"],
    outputs="number",
    title="Letter Counter",
    description="Enter text and a letter to count how many times the letter appears in the text."
)

# Launch both the Gradio web interface and the MCP server
if __name__ == "__main__":
    demo.launch(mcp_server=True)


/Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[11/10/25 09:56:54] INFO     StreamableHTTP session manager started                  ]8;id=943945;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=707403;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#112\112]8;;\

* Running on local URL:  http://127.0.0.1:7860


                    INFO     HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events      ]8;id=697446;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=973854;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py#1025\1025]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"            ]8;id=386494;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=558025;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py#1025\1025]8;;\

* To create a public link, set `share=True` in `launch()`.

🔨 Launching MCP server:
** Streamable HTTP URL: http://127.0.0.1:7860/gradio_api/mcp/
* [Deprecated] SSE URL: http://127.0.0.1:7860/gradio_api/mcp/sse


                    INFO     HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK" ]8;id=402021;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=993739;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py#1025\1025]8;;\

## How it works behind the scenes

When you set `mcp_server=True` in `launch()`, several things happen:
1. Gradio functions are automatically converted to MCP Tools
1. Input components determine the response format
1. The Gradio server now also listens for MCP protocol messages
1. JSON-RPC over HTTP+SSE is set up for client-server communication

## Key Features of the Gradio MCP Integration
1. **Tool Conversion:** Each API endpoint in your gradio app is automatically converted into an MCP tool with a corresponding name, description, and input schema. To view the tools and schemas, visit http://your-server:port/gradio_api/mcp/schema or go to the “View API” link in the footer of your Gradio app, and then click on “MCP”.
1. **Environment Variable Support:** There are two ways to enable the MCP server functionality:
* Using the `mcp_server` parameter in `launch()`:

In [12]:
demo.launch(mcp_server=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----


[11/10/25 09:57:00] INFO     HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"            ]8;id=570206;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=183112;file:///Users/christiancamp/Desktop/hugging-face-notes/hugging-env/lib/python3.13/site-packages/httpx/_client.py#1025\1025]8;;\

* To create a public link, set `share=True` in `launch()`.

🔨 Launching MCP server:
** Streamable HTTP URL: http://127.0.0.1:7860/gradio_api/mcp/
* [Deprecated] SSE URL: http://127.0.0.1:7860/gradio_api/mcp/sse


* Using environment variables:

```export GRADIO_MCP_SERVER=True```

3. **File Handling:** The server automatically handles file data conversions, including:
* Conversing base64-encoded strings to file data
* Processing image files and returning them in the correct format
* Managing temporary file storage

It is **strongly** recommended that input images and files be passed as full URLs as MCP Clients do not always handle local files correctly.

4. Hosted MCP Servers on 🤗 Spaces: You can publish your Gradio application for free on Hugging Face Spaces, which will allow you to have a free hosted MCP server. Here’s an example of such a Space: https://huggingface.co/spaces/abidlabs/mcp-tools

# Building and End-to-End MCP Application

In this unit, we’ll build a complete MCP application from scratch, focusing on creating a server with Gradio and connecting it with multiple clients. This hands-on approach will give you practical experience with the entire MCP ecosystem.